<a href="https://colab.research.google.com/github/codeson101001/Stock-Prediction-Models/blob/master/Stock_Investing_Strategy_3M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  import numpy as np
  import pandas as pd
  import matplotlib.pyplot as plt
  import seaborn as sns
  sns.set()
  !pip install yfinance
  import yfinance as yf 

In [ ]:
testing_multiple_stocks()

AAPL
[*********************100%***********************]  1 of 1 completed
day 1: buy 113 units at price 9946.259621, total balance 53.740379
day 3, sell 113 units at price 9936.372810, investment -0.099402 %, total balance 9990.113190,
day 5: buy 111 units at price 9958.642365, total balance 31.470825
day 7, sell 111 units at price 9991.664932, investment 0.331597 %, total balance 10023.135757,
day 8: buy 109 units at price 9941.889900, total balance 81.245857
day 9, sell 109 units at price 9636.417633, investment -3.072577 %, total balance 9717.663490,
day 10: buy 107 units at price 9677.614967, total balance 40.048523
day 13, sell 107 units at price 9739.942108, investment 0.644034 %, total balance 9779.990631,
day 14: buy 104 units at price 9720.100159, total balance 59.890472
day 19, sell 104 units at price 9929.660095, investment 2.155944 %, total balance 9989.550568,
day 20: buy 102 units at price 9899.865280, total balance 89.685287
day 22, sell 102 units at price 9845.294907, i

In [ ]:
def testing_multiple_stocks():

  starting = "2020-06-14"
  ending = "2020-09-14"

  stock_strategy("AAPL",starting,ending)
  stock_strategy("AMD",starting,ending)
  stock_strategy("AMZN",starting,ending)
  stock_strategy("GOOG",starting,ending)
  stock_strategy("MSFT",starting,ending)
  stock_strategy("TSLA",starting,ending)
  stock_strategy("NVDA",starting,ending)
  stock_strategy("MMM",starting,ending)
  stock_strategy("DELL",starting,ending)

In [ ]:
def stock_strategy(stock,starting_date,ending_date):
  print(stock)

  #df = yf.download(tickers="MMM", period="1y", interval="1d")  
  df = yf.download(stock, start = starting_date, end = ending_date)

  #This is how I calucalte part of my short and long window for the moving avg in the future
  #Depending on the number of days for our data we will determine the window for the MA's
  # If we have a large range of months from start to end we will get bigger number for the windows value
  # If we have a smaller range of months from start to end we will get smaller number for the windows value

  short_window = int(0.020 * len(df))
  long_window = int(0.06 * len(df))

  #We are storing the data from the stock information into signals
  signals = pd.DataFrame(index = df.index)

  #Here we are creating our dataframe column titles
  signals['price'] = df['Close']

  #We start by seting signals to 0.0 
  signals['signal'] = 0.0

  #Using df['Close'].rolling(_________).mean() is how we get our moving avgs
  #window: Size of the moving window. 
  #min_periods: Minimum number of observations in window required to have a value
  #center: helps with lagging the indicater 
  #https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html

  signals['short_ma'] = df['Close'].rolling(window=short_window, min_periods=1, center=False).mean()
  signals['long_ma'] = df['Close'].rolling(window=long_window, min_periods=1, center=False).mean()

  #Based on the short_ma and long_ma
  signals['signal'][short_window:] = np.where(signals['short_ma'][short_window:] > signals['long_ma'][short_window:], 1.0, 0.0) 

  #.diff() Calculates the difference of a Dataframe element compared with another element in the Dataframe (default is element in previous row).  
  signals['positions'] = signals['signal'].diff()

  signals







  #This is the stock simulator
  def buy_stock(real_movement, signal, initial_money = 10000, max_buy = 3500, max_sell = 3500):

      starting_money = initial_money
      states_sell = []
      states_buy = []
      current_inventory = 0

      #"i" is the day
      #"initial_money" is your money that you started out with
      #"real_movement" is the your money in the stock, which can be larger or smaller that the initial_money 
      def buy(i, initial_money, current_inventory):
          #Here we calualate the number of shares we want to buy using floor division, beucase we can not get pieces of a stock share
          shares = initial_money // real_movement[i]

          if shares < 1:
              print('day %d: total balances %f, not enough money to buy a unit price %f'% (i, initial_money, real_movement[i]))

          else:

              if shares > max_buy:
                  buy_units = max_buy

              else:
                  buy_units = shares
              initial_money -= buy_units * real_movement[i]
              current_inventory += buy_units
              print('day %d: buy %d units at price %f, total balance %f'% (i, buy_units, buy_units * real_movement[i], initial_money))
              states_buy.append(0)

          return initial_money, current_inventory



      #This will run for all the dates in in the df 
      for i in range(real_movement.shape[0] - int(0.025 * len(df))):
          state = signal[i]

          if state == 1:
              initial_money, current_inventory = buy(i, initial_money, current_inventory)
              states_buy.append(i)

          elif state == -1:

              if current_inventory == 0:
                      print('day %d: cannot sell anything, inventory 0' % (i))

              else:
                
                  if current_inventory > max_sell:
                      sell_units = max_sell

                  else:
                      sell_units = current_inventory
                  current_inventory -= sell_units
                  total_sell = sell_units * real_movement[i]
                  initial_money += total_sell

                  try:
                      invest = ((real_movement[i] - real_movement[states_buy[-1]])/ real_movement[states_buy[-1]]) * 100

                  except:
                      invest = 0
                  print('day %d, sell %d units at price %f, investment %f %%, total balance %f,'% (i, sell_units, total_sell, invest, initial_money))


              states_sell.append(i)



      invest = ((initial_money - starting_money) / starting_money) * 100
      total_gains = initial_money - starting_money
      return states_buy, states_sell, total_gains, invest

  states_buy, states_sell, total_gains, invest = buy_stock(df.Close, signals['positions'])

  print('total gains %f, total investment %f%%'%(total_gains, invest))




In [ ]:
#Large window
stock = "FB"
starting_date = "2020-01-14"
ending_date = "2020-12-14"

df = yf.download(stock, start = starting_date, end = ending_date)

short_window = int(0.030 * len(df))
long_window = int(0.06 * len(df))

print(short_window)
print(long_window)
print('Number of trading days',len(df))

[*********************100%***********************]  1 of 1 completed
5
10
Number of trading days 169


In [ ]:
#Small window
stock = "FB"
starting_date = "2020-07-14"
ending_date = "2020-09-14"

df = yf.download(stock, start = starting_date, end = ending_date)

short_window = int(0.030 * len(df))
long_window = int(0.06 * len(df))

print(short_window)
print(long_window)
print('Number of trading days',len(df))

[*********************100%***********************]  1 of 1 completed
1
2
Number of trading days 43
